<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EB%8C%80%EB%B6%84%EB%A5%98_spell_check%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_G_inc.csv		   df_P_inc.csv
'Colab Notebooks'	 df_H_inc_노이.csv	  'My Drive'
 C_ori_model.pt		 df_J_inc_again_노이.csv   대분류모델.pt
'답안 작성용 파일.csv'	 df_J_inc.csv		   Q_inc_model.pt
 df_C_inc.csv		 df_J_inc_dain.csv	   spell_check
'# df_G_inc.csv'	 df_K_inc.csv


In [ ]:
!unzip '/content/gdrive/MyDrive/산업분류자동화/모델 개발용 자료.zip' -d /content/dataset

Archive:  /content/gdrive/MyDrive/산업분류자동화/모델 개발용 자료.zip
2. 모델개발용자료.txt:  mismatching "local" filename (2. ыкиыН╕ъ░Ьы░ЬьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/2. 모델개발용자료.txt  


In [ ]:
import pandas as pd
df=pd.read_csv('/content/dataset/2. 모델개발용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_000001,NaN,NaN,NaN,치킨전문점에서 고객의주문에의해 치킨판매
1,id_000002,NaN,NaN,NaN,산업공구 다른 소매업자에게 철물 수공구
2,id_000003,NaN,NaN,NaN,절에서 신도을 대상으로 불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서 고객요구로 자동차튜닝
4,id_000005,NaN,NaN,NaN,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공"
...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서 일반인대상으로 버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서 외래환자위주고 치료
99997,id_099998,NaN,NaN,NaN,일반점포에서 소비자에게 그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서 일반인.학생대상으로 학습공간제공


## 맞춤법 교정

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-s9fbirem
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-s9fbirem
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=fd033c142dcf04242d51c37057ac244451d5c1153584655cf117fcd3d9973c1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-zslcqwqn/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [ ]:
df.to_csv('ts_spellcheck.csv', index=False, encoding='utf-8-sig')